In [25]:
import prody
import openenm as oenm
import numpy as np

## OpenENM GNM

In [20]:
gnm_oenm = oenm.GaussianNetworkModel('1tcd', cutoff='10 angstroms')

In [21]:
gnm_oenm.kirchhoff_matrix

array([[11, -1, -1, ...,  0,  0,  0],
       [-1, 14, -1, ...,  0,  0,  0],
       [-1, -1, 14, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ..., 19, -1, -1],
       [ 0,  0,  0, ..., -1, 22, -1],
       [ 0,  0,  0, ..., -1, -1, 13]])

## Prody GNM

In [3]:
tri = prody.parsePDB('1tcd')

@> Connecting wwPDB FTP server RCSB PDB (USA).
@> 1tcd downloaded (1tcd.pdb.gz)
@> PDB download via FTP completed (1 downloaded, 0 failed).
@> 3983 atoms and 1 coordinate set(s) were parsed in 0.07s.


In [6]:
calphas = tri.select('calpha')

In [13]:
gnm_prody = prody.GNM('Triosa')

### Kirchhoff matrix comparison

In [23]:
gnm_prody.buildKirchhoff(calphas, cutoff=10., gamma=1.)

@> Kirchhoff was built in 0.03s.


In [26]:
gnm_prody.getKirchhoff()

array([[11., -1., -1., ...,  0.,  0.,  0.],
       [-1., 14., -1., ...,  0.,  0.,  0.],
       [-1., -1., 14., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ..., 19., -1., -1.],
       [ 0.,  0.,  0., ..., -1., 22., -1.],
       [ 0.,  0.,  0., ..., -1., -1., 13.]])

In [27]:
np.all(gnm_oenm.kirchhoff_matrix==gnm_prody.getKirchhoff())

True

### Eigvals and Eigvects comparison

In [36]:
gnm_prody.calcModes('all', zeros=True)

@> 497 modes were calculated in 0.13s.


In [39]:
np.allclose(gnm_oenm.eigenvalues, gnm_prody.getEigvals())

True

In [45]:
np.allclose(gnm_oenm.eigenvectors, gnm_prody.getEigvecs())

False

In [43]:
eigvecs_prody = gnm_prody.getEigvecs()

In [49]:
eigvecs_prody[0][:10]

array([-0.04485613, -0.05756788,  0.02853722,  0.00827805,  0.03945476,
       -0.01531639, -0.06829124, -0.0678335 , -0.00783387,  0.00371932])

In [50]:
gnm_oenm.eigenvectors[0][:10]

array([ 0.04485613, -0.05756788,  0.02853722, -0.00827805,  0.03945476,
       -0.01531639,  0.06829124, -0.0678335 ,  0.00783387, -0.00371932])

In [71]:
comparison=[]
for ii in range(497):
    if np.sign(gnm_oenm.eigenvectors[0][ii])!=np.sign(eigvecs_prody[0][ii]):
        comparison.append(np.allclose(gnm_oenm.eigenvectors[:,ii], -eigvecs_prody[:,ii]))
    else:
        comparison.append(np.allclose(gnm_oenm.eigenvectors[:,ii], eigvecs_prody[:,ii]))

In [73]:
np.all(comparison)

True